In [1]:
from datetime import date
import yfinance as yf
import pandas as pd
import warnings


# Suppress future warnings
warnings.filterwarnings('ignore', category=FutureWarning)

# 1. Introduction



In [2]:
stock_ticker = 'MSFT'

# 2. Fetching the Data
Typically, the `yfinance` module can go back as far back in time as the historical data of a stock in the market. For stocks like microsoft and Apple, this could mean over 35 years of data. This is not in itself a problem, except for the fact that in the models that we need to develop, we want to build in two exogeneous variables to help with predictions:
* The daily price of  TIP, which started trading on 5th December 2003.
* The Microsoft Earnings calls, which go back as far as 26th January 2015. These calls have taken place quarterly since the company first went public in 1986; however, the data for the estimated and actual earnings per share is not available on their site prior to 2015.


## 2.1. Extracting Daily Closing Prices for a TIPS

* The longest-running ETF that tracks Treasury Inflation-Protected Securities (TIPS) is the iShares TIPS Bond ETF (ticker: **TIP**).
* It was launched by BlackRock on December 4, 2003.

* This ETF aims to track the investment results of an index composed of inflation-protected U.S. Treasury bonds.
* It's one of the most well-known and widely traded TIPS ETFs, providing investors with exposure to inflation-protected bonds issued by the U.S. government.

In [3]:
# Choose the ticker symbol
tickerSymbol = 'TIP'

# Get data on this ticker
tickerData = yf.Ticker(tickerSymbol)

# Get the historical prices for this ticker
tip = tickerData.history(period="max")

# Extract the Closing Price only
tip = pd.DataFrame(tip['Close'])

# Rename the Close column for clarity
tip.rename(columns = {'Close': 'tip_price'}, inplace = True)

# Make the date index timezone-naive since the earnings call data will be as well
tip.index = [d.date() for d in tip.index.to_pydatetime()]
# Visualise
tip

,tip_price
2003-12-05,53.091015
2003-12-08,53.044102
2003-12-09,52.908573
2003-12-10,53.049324
2003-12-11,53.257805
...,...
2024-07-29,107.338074
2024-07-30,107.417938
2024-07-31,108.006996
2024-08-01,108.150002


## 2.2. Extracting the Data on Earnings Calls

The data on the Earnings Calls for Microsoft, including: their dates, the estimated Earnings-per-Share (EPS) and the Actual EPS, can be found on [this website](https://www.alphaquery.com/stock/MSFT/earnings-history).

In [4]:
import requests
import bs4

In [5]:
# # Install Selenium if not already
# pip install selenium
# !pip install webdriver-manager


In [6]:
# # Install chromium, its driver, and selenium
# !apt-get update
# !apt install -y chromium-chromedriver


In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

# Correct path to the ChromeDriver executable
service = Service(r'C:\Program Files\chromedriver-win64\chromedriver.exe')  # Replace with the actual path

driver = webdriver.Chrome(service=service)

# Define the URL
url = f'https://www.alphaquery.com/stock/{stock_ticker}/earnings-history'

# Navigate to the URL
driver.get(url)

# Wait for the page to fully load (you can adjust the sleep time if needed)
driver.implicitly_wait(10)

# Find the table element
table = driver.find_element(By.TAG_NAME, 'table')

# Extract the table rows
rows = table.find_elements(By.TAG_NAME, 'tr')

# Initialize a list to store the table data
table_data = []

# Loop through the rows and extract the data
for row in rows:
    cells = row.find_elements(By.TAG_NAME, 'td')
    cell_data = [cell.text for cell in cells]
    table_data.append(cell_data)

# Close the WebDriver
driver.quit()

# Convert the data to a pandas DataFrame
earnings_calls = pd.DataFrame(table_data)

# Now let's reformat the df for it to be meaningful
earnings_calls.drop(0, inplace = True) # The first row is an empty artefact of the scraping and should be removed
# Next we will sort through the date so that the ascending order matches
# the order of the Price dataframe that will be generated by yfinance
earnings_calls.sort_values(by = 0, inplace = True)
earnings_calls.reset_index(drop=True, inplace=True) #Reset the index since we removed a row and resorted
earnings_calls.drop(1, axis = 1, inplace = True) #Removes column '1' as this is the Date of the Fiscal Quarter End that the Earnings Call relates to.
# We are looking at daily events and how they influence *future* prices, so this data is not relevant.

# Lets also rename the columns to their original ehadings
earnings_calls.rename(columns = {0: 'earnings_call_date', 2: "estimated_eps", 3: "actual_eps"}, inplace = True)


# Cast the Date column to a Datetime object since it will be easier when we want to merge dataframes 
# later with the yfinance data
earnings_calls['earnings_call_date'] = pd.to_datetime(earnings_calls['earnings_call_date'])
# We want to make them timezone-naive
earnings_calls['earnings_call_date'] = earnings_calls['earnings_call_date'].dt.date

#Let's sate the dates column as the index since it will match the format of the data imported by yfinance
earnings_calls.set_index('earnings_call_date', inplace = True)

# Display the DataFrame
earnings_calls


,estimated_eps,actual_eps
earnings_call_date,,
2015-01-26,$0.70,$0.77
2015-04-23,$0.51,$0.62
2015-07-21,$0.56,$0.62
2015-10-22,$0.58,$0.67
2016-01-28,$0.69,$0.78
2016-04-21,$0.63,$0.62
2016-07-19,$0.58,$0.69
2016-10-20,$0.68,$0.76
2017-01-26,$0.79,$0.83


## 2.3. Extracting the Daily Closing Prices on MSFT
Just like we did for **TIP**, we will use the `yfinance` module to extract historical data on daily closing prices for the stock that we loaded on the 1st cell.

In [8]:
# Get data on this ticker
tickerData = yf.Ticker(stock_ticker) # Loads the financial data on whatever stock ticker we loaded on cell 1

# Get the historical prices for this ticker
stock = tickerData.history(period="max")
# Extract the Closing Price only
stock = pd.DataFrame(stock['Close'])

# Rename the Close column for clarity
stock.rename(columns = {'Close': 'stock_price'}, inplace = True)

# Make the date index timezone-naive since the earnings call data will be as well
stock.index = [d.date() for d in stock.index.to_pydatetime()]

# Visualise
stock

,stock_price
1986-03-13,0.060055
1986-03-14,0.062199
1986-03-17,0.063272
1986-03-18,0.061663
1986-03-19,0.060591
...,...
2024-07-29,426.730011
2024-07-30,422.920013
2024-07-31,418.350006
2024-08-01,417.109985


## 2.4. Merging the Three Dataframes according to their Shared Timeline


In [9]:
stock_start = stock.index[0]
tip_start = tip.index[0]
earnings_calls_start = earnings_calls.index[0]

In [10]:
print(stock_start)
print(type(stock_start))
print(tip_start)
print(type(tip_start))
print(earnings_calls_start)
print(type(earnings_calls_start))

1986-03-13
<class 'datetime.date'>
2003-12-05
<class 'datetime.date'>
2015-01-26
<class 'datetime.date'>


In [11]:
if stock_start < earnings_calls_start:
    print(True)

True


In [12]:
def mindful_concatenator (stock_df, inflation_df, earnings_calls_df):
    # First thing we want to do is to only keep the data they have from their shared timelines
    # We will fetch the df that starts the latest and filter the others to start at that same date.
    latest_date = max(stock_df.index[0], inflation_df.index[0], earnings_calls_df.index[0])
    # Now that I have obtained the 'earliest' date they've got in common, I will filter them
    new_stock = stock[latest_date:] #Restrict
    new_tip = tip[latest_date:]
    new_ec = earnings_calls[latest_date:]
    # The inflation and stock dataframes should have the same trading dates - bar some market events that
    # might have required one of the two tickers to stop trading for a period of time.
    # In case this has happened, I will do an inner join to ensure they have the same trading dates.
    inflation_with_stock_dfs = pd.concat([new_tip, new_stock], axis = 1, join = 'inner')
    # The Earnings Calls data is different - these calls only happen quarterly, so I want there to be a 0
    # on any trading date which does not have an earnings call.
    combined_df = pd.concat([inflation_with_stock_dfs, new_ec], axis = 1, join = 'outer')
    # This would have generated a NaN on every trading date without an earnings call.
    return combined_df


In [13]:
final = mindful_concatenator(stock, tip, earnings_calls)

In [16]:
final

,tip_price,stock_price,estimated_eps,actual_eps
2015-01-26,89.041435,40.520855,$0.70,$0.77
2015-01-27,89.322266,36.771320,NaN,NaN
2015-01-28,89.665565,35.504242,NaN,NaN
2015-01-29,89.564163,36.211052,NaN,NaN
2015-01-30,90.211700,34.823292,NaN,NaN
...,...,...,...,...
2024-07-29,107.338074,426.730011,NaN,NaN
2024-07-30,107.417938,422.920013,$2.90,$2.95
2024-07-31,108.006996,418.350006,NaN,NaN
2024-08-01,108.150002,417.109985,NaN,NaN


In [ ]:
data.set_index('date', inplace = True)
data

In [ ]:
from datetime import datetime
filterdate = '2023-02-01'
filterdate = datetime.strptime(filterdate, '%Y-%m-%d').date()


data[filterdate:]

In [ ]:
end_date = date.today()
print(end_date)
end_date_str = end_date.strftime('%Y-%m-%d')
print(end_date_str)

In [ ]:
start_date = '2015-01-26'

In [ ]:
# Choose the ticker symbol
tickerSymbol = 'MSFT'

# Get data on this ticker
tickerData = yf.Ticker(tickerSymbol)

# Get the historical prices for this ticker
# We want to load the data up until the 10th July - we will restrict the dataframe to the 3rd July later
msft = tickerData.history(period='1d', start = start_date, end=end_date) #We need to add 1 day to the end of the range so that the time range goes up to the desired target date

# Visualise
msft

Typically, the `yfinance` module can go back as far back in time as the historical data of a stock in the market. For stocks like microsoft and Apple, this could mean over 35 years of data. This is not in itself a problem, except for the fact that in the models that we need to develop, we want to build in two exogeneous variables to help with predictions:
* The daily price of a TIP, which started trading on 5th December 2003.
* The Microsoft Earnings calls, which go back as far as 26th January 2015. These calls have taken place quarterly since the company first went public in 1986; however, the data for the estimated and actual earnings per share is not available on their site prior to 2015.
